In [0]:
# Adapted from : https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
'''
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, *datasets):
        self.datasets = datasets

    def __getitem__(self, i):
      if i<len(self.datasets[0]):
        return self.datasets[0][i]
      else :
        return self.datasets[1][len(self.datasets[0])-i]
        #return tuple(d[i] for d in self.datasets)

    def __len__(self):
        return sum(len(d) for d in self.datasets)
data1 = ConcatDataset(datasets.ImageFolder(args["data"] + '/train_images',
                         transform=data_transforms_train),
                 datasets.ImageFolder(args["data"] + '/val_images',
                        transform=data_transforms_val))
validation_split=0.3
random_seed= 42
shuffle_dataset=True
dataset_size = len(data1)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader1 = torch.utils.data.DataLoader(data1, batch_size=args["batch_size"], 
                                           sampler=train_sampler)
val_loader1 = torch.utils.data.DataLoader(data1, batch_size=args["batch_size"],
                                                sampler=valid_sampler)

set_loader = torch.utils.data.DataLoader(
             ConcatDataset(
                 datasets.ImageFolder(args["data"] + '/train_images',
                         transform=data_transforms_train),
                 datasets.ImageFolder(args["data"] + '/val_images',
                         transform=data_transforms_val)),
             batch_size=args["batch_size"], shuffle=True,
             num_workers=1)
#, pin_memory=True)
'''

'\nclass ConcatDataset(torch.utils.data.Dataset):\n    def __init__(self, *datasets):\n        self.datasets = datasets\n\n    def __getitem__(self, i):\n      if i<len(self.datasets[0]):\n        return self.datasets[0][i]\n      else :\n        return self.datasets[1][len(self.datasets[0])-i]\n        #return tuple(d[i] for d in self.datasets)\n\n    def __len__(self):\n        return sum(len(d) for d in self.datasets)\ndata1 = ConcatDataset(datasets.ImageFolder(args["data"] + \'/train_images\',\n                         transform=data_transforms_train),\n                 datasets.ImageFolder(args["data"] + \'/val_images\',\n                        transform=data_transforms_val))\nvalidation_split=0.3\nrandom_seed= 42\nshuffle_dataset=True\ndataset_size = len(data1)\nindices = list(range(dataset_size))\nsplit = int(np.floor(validation_split * dataset_size))\nif shuffle_dataset :\n    np.random.seed(random_seed)\n    np.random.shuffle(indices)\ntrain_indices, val_indices = indices[spl

In [0]:
# once the images are loaded, how do we pre-process them before being passed into the network
# by default, we resize the images to 64 x 64 in size
# and normalize them to mean = 0 and standard-deviation = 1 based on statistics collected from
# the training set

data_transforms = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

data_transforms_train = transforms.Compose([
        #transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

data_transforms_val = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [0]:
#### Importing the datasets 

dataset = '/content/drive/My Drive/recvis19_a3-master/bird_dataset'
batch_size = 20

train_datasets = datasets.ImageFolder(dataset + '/train_images',
                         transform=data_transforms_train)

train_loader = torch.utils.data.DataLoader(train_datasets, batch_size=batch_size, shuffle=True, num_workers=1)

val_datasets =  datasets.ImageFolder(dataset + '/val_images',transform=data_transforms_val)
val_loader = torch.utils.data.DataLoader(val_datasets,batch_size=batch_size, shuffle=False, num_workers=1)


dataloaders = {'train':train_loader, 'val':val_loader}
dataset_sizes = {'train':len(train_datasets), 'val':len(val_datasets)}

class_names_train = train_datasets.classes

In [0]:
def train(epoch):

  model.train()
  correct = 0
  for batch_idx, (data, target) in enumerate(train_loader):
    if use_cuda:
      data, target = data.cuda(), target.cuda()
      optimizer.zero_grad()
      output = model(data)
      criterion = torch.nn.CrossEntropyLoss(reduction='mean')
      loss = criterion(output, target)
      loss.backward()
      optimizer.step()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).cpu().sum()
      if batch_idx % log_interval == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item()), 'Accuracy: {}/{} ({:.0f}%)'.format(
        correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset))) 

def validation():

    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        # sum up batch loss
        criterion = torch.nn.CrossEntropyLoss(reduction='mean')
        validation_loss += criterion(output, target).data.item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))
    

In [0]:
CUDA_LAUNCH_BLOCKING=1

import argparse
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torch.autograd import Variable
from tqdm import tqdm
import easydict
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import torchvision.models as models
from torch.optim import lr_scheduler

nclasses = 20
data = '/content/gdrive/My Drive/recvis/bird_dataset'
batch_size =20
epoch =  20
lr =  0.1
momentum = 0.9
seed = 1
log_interval = 10
experiment = '/content/gdrive/My Drive/recvis/experiment'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

use_cuda = torch.cuda.is_available()
torch.manual_seed(seed)

# Create experiment folder
if not os.path.isdir(experiment):
    os.makedirs(experiment)

# Data initialization and loading
#from data import data_transforms

# Neural network and optimizer
# We define neural net in model.py so that it can be reused by the evaluate.py script

def model_choice(name='resnet152'):
  if name =='vgg19_bn':
    model_ft = models.vgg19_bn(pretrained='imagenet')
    model_conv = nn.Sequential(*list(model.children())[:-1])
    for param in model_conv.parameters():
        param.requires_grad = False
    model_ft.classifier = nn.Sequential(
        nn.Linear(512 * 7 * 7, 4096),
        nn.ReLU(True),
        nn.Dropout(),
        nn.Linear(4096, 4096),
        nn.ReLU(True),
        nn.Dropout(),
        nn.Linear(4096, nclasses),
    )

  else:
    if name=='resnet152':
      model_ft = models.resnet152(pretrained=True)
      num_ftrs = model_ft.fc.in_features
      model_ft.fc = nn.Linear(num_ftrs, 20)
    if name=='resnet101':
      model_ft = models.resnet101(pretrained=True)
      num_ftrs = model_ft.fc.in_features
      model_ft.fc = nn.Linear(num_ftrs, 20)
    elif name=='resnet18':
      model_ft = models.resnet18(pretrained=True)
      num_ftrs = model_ft.fc.in_features
      model_ft.fc = nn.Linear(num_ftrs, 20)
    else:
      model_ft = models.resnet50(pretrained=True)
      num_ftrs = model_ft.fc.in_features
      model_ft.fc = nn.Linear(num_ftrs, 20)
      
  return model_ft.to(device)

name = 'resnet101'
model = model_choice(name=name)
'''
for name, child in model.named_children():
   if name in ['fc','layer4','layer3']:
       print(name + ' is unfrozen')
       for param in child.parameters():
           param.requires_grad = True
   else:
       print(name + ' is frozen')
       for param in child.parameters():
           param.requires_grad = False
'''
if use_cuda:
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU')

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

#best_acc = 0
for epoch in range(1, epoch + 1):
    train(epoch)
    validation()
    #acc = 100. * correct / len(val_loader.dataset)
    #if acc > best_acc : 
      #best_acc = acc
      #best_model = copy.deepcopy(model.state_dict())     
    model_file = experiment + '/model_' + str(epoch) + '.pth'
    torch.save(model.state_dict(), model_file)
    print('Saved model to ' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file\n')

#model.load_state_dict(best_model)

cuda:0


Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/checkpoints/resnet101-5d3b4d8f.pth
100%|██████████| 170M/170M [00:02<00:00, 66.3MB/s]


Using GPU
Train Epoch: 1 [0/1090 (0%)]	Loss: 3.136143 Accuracy: 0/1090 (0%)
Train Epoch: 1 [200/1090 (18%)]	Loss: 3.067652 Accuracy: 10/1090 (1%)
Train Epoch: 1 [400/1090 (36%)]	Loss: 2.743560 Accuracy: 32/1090 (3%)
Train Epoch: 1 [600/1090 (55%)]	Loss: 2.642950 Accuracy: 85/1090 (8%)
Train Epoch: 1 [800/1090 (73%)]	Loss: 2.373545 Accuracy: 163/1090 (15%)
Train Epoch: 1 [1000/1090 (91%)]	Loss: 2.001949 Accuracy: 253/1090 (23%)

Validation set: Average loss: 0.0972, Accuracy: 66/103 (64%)
Saved model to /content/gdrive/My Drive/recvis/experiment/model_1.pth. You can run `python evaluate.py --model /content/gdrive/My Drive/recvis/experiment/model_1.pth` to generate the Kaggle formatted csv file

Train Epoch: 2 [0/1090 (0%)]	Loss: 2.049935 Accuracy: 12/1090 (1%)
Train Epoch: 2 [200/1090 (18%)]	Loss: 1.693794 Accuracy: 137/1090 (13%)
Train Epoch: 2 [400/1090 (36%)]	Loss: 1.409563 Accuracy: 271/1090 (25%)
Train Epoch: 2 [600/1090 (55%)]	Loss: 1.597973 Accuracy: 409/1090 (38%)
Train Epoch: 2

In [0]:
import argparse
from tqdm import tqdm
import os
import PIL.Image as Image

import torch

parser = argparse.ArgumentParser(description='RecVis A3 evaluation script')
parser.add_argument('--data', type=str, default='bird_dataset', metavar='D',
                    help="folder where data is located. test_images/ need to be found in the folder")
parser.add_argument('--model', type=str, metavar='M',
                    help="the model file to be evaluated. Usually it is of the form model_X.pth")
parser.add_argument('--outfile', type=str, default='experiment/kaggle.csv', metavar='D',
                    help="name of the output csv file")


path_read = '/content/drive/My Drive/recvis19_a3-master/bird_dataset'
path_write = '/content/drive/My Drive/recvis19_a3-master/experiment'
outfile = '/content/drive/My Drive/recvis19_a3-master/experiment/kaggle.csv'
seed = 1
momentum = 0.5
batch_size = 64
epochs = 10
lr = 0.1
log_interval = 10

use_cuda = torch.cuda.is_available()

state_dict = torch.load('/content/gdrive/My Drive/recvis/experiment/model_12.pth')
model = models.resnet101(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs,20)
model.load_state_dict(state_dict)
model.eval()
if use_cuda:
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU')


test_dir = path_read + '/test_images/mistery_category'

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')


output_file = open('/content/drive/My Drive/recvis19_a3-master/experiment/kaggle101neww.csv', "w")
output_file.write("Id,Category\n")
for f in tqdm(os.listdir(test_dir)):
    if 'jpg' in f:
        data = data_transforms(pil_loader(test_dir + '/' + f))
        data = data.view(1, data.size(0), data.size(1), data.size(2))
        if use_cuda:
            data = data.cuda()
        output = model(data)
        pred = output.data.max(1, keepdim=True)[1]
        output_file.write("%s,%d\n" % (f[:-4], pred))

output_file.close()

print("Succesfully wrote " + outfile + ', you can upload this file to the kaggle competition website')

Using GPU


100%|██████████| 517/517 [03:55<00:00,  1.92it/s]

Succesfully wrote /content/drive/My Drive/recvis19_a3-master/experiment/kaggle.csv, you can upload this file to the kaggle competition website
